<a href="https://colab.research.google.com/github/MichaelPaulukonis/notebooks/blob/main/min_dalle_(version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# min(DALL·E)

In [ ]:
#@title Save to Google Drive
#@markdown Will save your results to Google Drive - image and prompt+settings

save_to_google_drive = True  #@param {type: "boolean"}

if save_to_google_drive:
  from google.colab import drive
  import time
  
  #@title Google Colab Google Drive Downloader Thing { vertical-output: true }
  Target_Folder = "/content/drive/MyDrive/colab_out/" #@param {type:"string"}
  drive.mount('/content/drive')
  
  !mkdir -p $Target_Folder

  save_prompt = True  #@param {type: "boolean"}
  prompt_file = "/content/drive/MyDrive/colab_out/prompts.txt" #@param {type:"string"}


### Install

In [ ]:
! nvidia-smi
! pip install min-dalle==0.4.7
! pip install Pillow

### Load Model
`float32` is faster than `float16` but uses more GPU memory.

In [ ]:
dtype = "float16" #@param ["float32", "float16"]
from IPython.display import display, update_display
import torch
from min_dalle import MinDalle

model = MinDalle(
    dtype=getattr(torch, dtype),
    device='cuda',
    is_mega=True, 
    is_reusable=True
)

### Generate Images

- `image_prefix` name of the images (if you're saving to Google Drive)

- `progressive_outputs` Whether to show intermediate output.  Adds a small delay and increases memory usage.

- `seamless` Tiles the images in token space instead of pixel space

- `grid_size` Size of the image grid.  Reduce this if you run out of GPU memory.

- `batch_size` Number of images to generate. If you're not saving to GDrive, this is somewhat pointless.

- `temperature` High temperature increases the probability of sampling low scoring image tokens.

- `supercondition_factor` Higher values result in better agreement with the text but a narrower variety of generated images

- `top_k` Each image token is sampled from the top $k$ most probable tokens

- `seed_behavior` Random creates a new random number for each image in batch. Iter will increment by 1. Fixed will stay the same. Really only useful if you add extra coding to modify other params in batch mode.

- `seed` Using the built-in random seed results in different images each time. Set manually to get results starting from the same seed. An integer from 1 to 2**32 - 1. Use -1 for random.

- `is_verbose` min-dalle will print out extra internal data (mostly about text tokenization)

In [ ]:
%%time
from types import SimpleNamespace

def DalleArgs():
  text = "anatomical medical illustration chart diagram+screenprint by robert rauschenberg+strong dayglo colors" #@param {type:"string"}
  if save_to_google_drive:
    image_prefix = "rauschenberg.1016" #@param {type:"string"}
  progressive_outputs = False #@param {type:"boolean"}
  seamless = False #@param {type:"boolean"}
  grid_size =  3 #@param {type:"integer"}
  batch_size = "2" #@param [1,2,3,4,5,6,7,8,9,10,15,20]
  temperature = 2.17 #@param {type:"slider", min:0.01, max:16, step:0.01}
  supercondition_factor =  "64" #@param [4, 8, 16, 32, 64]
  top_k =  "4" #@param [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]
  seed_behavior = "iter" #@param ["iter","fixed","random"]
  seed = -1 #@param {type:"integer"}
  is_verbose = False #@param {type:"boolean"}

  batch_size = int(batch_size)

  return locals()

args_dict = DalleArgs()
args = SimpleNamespace(**args_dict)

if save_to_google_drive:
  !mkdir -p '{Target_Folder}/{args.image_prefix}'

if args.seed == -1:
    args.seed = torch.randint(0,2**32 - 1,(1,))[0].item()

def next_seed(args):
    if args.seed_behavior == 'iter':
        args.seed += 1
    elif args.seed_behavior == 'fixed':
        pass # always keep seed the same
    else:
        args.seed = torch.randint(0,2**32 - 1,(1,))[0].item()
    return args.seed

if save_prompt:
  with open(prompt_file, 'a') as f:
    f.write(f'{args.text} {{ temp: {args.temperature}, sf: {args.supercondition_factor} top_k: {args.top_k}, seed: {args.seed}, start: {time.strftime("%Y%m%d-%H%M%S")} }}\n')

for x in range(0, args.batch_size):
  print(f"{x+1} of {args.batch_size} temp: {args.temperature} seed: {args.seed}")

  image = model.generate_image(
      text=args.text,
      seed=args.seed,
      grid_size=args.grid_size,
      is_seamless=args.seamless,
      temperature=args.temperature,
      top_k=int(args.top_k),
      supercondition_factor=float(args.supercondition_factor),
      is_verbose=args.is_verbose
  )

  display(image)

  if save_to_google_drive:
    timestr = time.strftime("%Y%m%d-%H%M%S")
    image.save(f'{Target_Folder}/{args.image_prefix}/{args.image_prefix}_{timestr}_{args.temperature}_{args.supercondition_factor}_{args.top_k}.png')

  seed = next_seed(args)
  